In [4]:
import requests 

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from pprint import pprint
from time import sleep
import pandas as pd
import numpy as np

In [9]:
def extrae_datos_meneos (lista_meneos, año_busqueda):
    datos = []
    continuar = True
    for meneo in lista_meneos:
        dicc_aux = {}
        
        # Se obtiene en primer lugar el año
        try:
            fecha_pub = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")[1].text.split(" ")[0]
            año = datetime.strptime(fecha_pub,"%d/%m/%Y").year if len(fecha_pub) > 5 else 2024
        except:
            año = False
        
        # Si el año detectado es igual que el año buscado, se continúa con la importación
        if año == año_busqueda:

            ###################################   Comunidad (Temática) ############################################################
            try:
                dicc_aux["Comunidad"] = meneo.find("a", class_="subname").text.strip()
            except:
                dicc_aux["Comunidad"] = np.nan

            ###################################    Usuario  ######################################################################
            try:
                dicc_aux["Usuario"] = meneo.find("div", class_="news-submitted").find("img")["alt"].strip()
            except:
                dicc_aux["Usuario"] = np.nan

            ###################################    Medio    ######################################################################
            try:
                dicc_aux["Medio"] = meneo.find("span", class_="showmytitle").text.strip()
            except:            
                dicc_aux["Medio"] = "meneame.net"

            ###################################    URL    ######################################################################
            try:
                dicc_aux["URL"] = meneo.find("span", class_="showmytitle")["title"].strip()
            except:            
                dicc_aux["URL"] = "https://www.meneame.net" + meneo.find("a", class_="comments")["href"].strip()

            ###################################    Fechas    ######################################################################    
            try:
                fechas = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")
                dicc_aux["Enviado"] = fechas[0].text.strip()
                dicc_aux["Publicado"] = fechas[1].text.strip()
            except:
                dicc_aux["Enviado"] = np.nan
                dicc_aux["Publicado"] = np.nan

            ###################################    Titular    ####################################################################
            try:
                dicc_aux["Titular"] = meneo.find("h2").text.replace("\n", " ").strip()
            except:
                dicc_aux["Titular"] = np.nan

            ###################################    Entradilla    ###################################################################
            try:
                dicc_aux["Entradilla"] = meneo.find("div", class_="news-content").text.replace("\n", " ").strip()
            except:
                dicc_aux["Entradilla"] = np.nan
        
            ###################################    Votos    ######################################################################
            try:
                dicc_aux["Votos"] = meneo.find("div", class_="votes").find("a").text.strip()
            except:
                dicc_aux["Votos"] = np.nan

            ###################################    Click    ######################################################################
            try:
                dicc_aux["Clicks"] = meneo.find("div", class_="clics").find("span").text.strip()
            except:
                dicc_aux["Clicks"] = np.nan

            ###################################    Positivos    ###################################################################
            try:
                dicc_aux["Positivos"] = meneo.find("span", class_="positive-vote-number").text.strip()
            except:
                dicc_aux["Positivos"] = np.nan

            ###################################    Votos Anónimos    ##############################################################
            try:
                dicc_aux["Anonimos"] = meneo.find("span", class_="anonymous-vote-number").text.strip()
            except:
                dicc_aux["Anonimos"] = np.nan

            ###################################    Votos Negativos    ##############################################################
            try:
                dicc_aux["Negativos"] = meneo.find("span", class_="negative-vote-number").text.strip()
            except:
                dicc_aux["Negativos"] = np.nan

            ###################################    Número comentarios    ###########################################################
            try:
                dicc_aux["Comentarios"] = meneo.find("a", class_="comments")["data-comments-number"].strip()
            except:
                dicc_aux["Comentarios"] = np.nan

            ###################################    Karma    ########################################################################
            try:
                dicc_aux["Karma"] = meneo.find("span", class_="karma-number").text.strip()
            except:
                dicc_aux["Karma"] = np.nan

            datos.append(dicc_aux)

        # Si se ha terminado el año buscado se detendrá la importación
        elif año < año_busqueda:
            continuar = False

    return [datos, continuar]

In [13]:
######################################## BLOQUE PARA PRUEBAS ##############################################################

# Para obtener todo el año 2023, empezar con la pagina 128 y eliminar la condición de "contador" en el bucle ####

######################################## BLOQUE PARA PRUEBAS ##############################################################

browser = webdriver.Chrome()
pagina = 737
lista_aux = []
contador = 0
continuar = True
año = 2023

while contador < 10 and continuar == True:
    try:
        browser.get(f"https://old.meneame.net/?page={pagina}")        
        Soup_pagina_meneos = BeautifulSoup(browser.page_source, "html.parser")
        lista_meneos = Soup_pagina_meneos.find_all("div", class_="news-summary")
        resultado = extrae_datos_meneos(lista_meneos, año)
        lista_aux.extend(resultado[0])
        continuar = resultado[1]
        pagina += 1
        contador += 1
    except:
        print(f"Error en página {pagina}")    

print(f"Total paginas procesadas: {contador}")
print(f"Total meneos importados: {len(lista_aux)}")

browser.close()

Total paginas procesadas: 2
Total meneos importados: 26


In [14]:
df = pd.DataFrame(lista_aux)
df

,Comunidad,Usuario,Medio,URL,Enviado,Publicado,Titular,Entradilla,Votos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma
0,actualidad,--557077--,climatica.lamarea.com,https://www.climatica.lamarea.com/edificio-ent...,01/01/2023 11:33,01/01/2023 19:00,Érase una vez un edificio con un submarino dentro,"En Madrid, diecisiete familias llevan dos años...",248,8732,122,126,0,15,488
1,ocio,Dakaira,youtube.com,https://www.youtube.com/watch?v=L7jEqG2_vYU,01/01/2023 12:41,01/01/2023 19:00,Marcha radetzky,La Marcha Radetzky fue compuesta por Johann St...,119,1279,63,56,0,14,492
2,ciencia,ccguy,danielmarin.naukas.com,https://danielmarin.naukas.com/2022/12/31/el-p...,31/12/2022 17:53,01/01/2023 18:25,El panorama espacial en 2022,2022 ha sido un año récord en el espacio. Se h...,150,2257,87,63,2,41,484
3,actualidad,Bonoman,linkedin.com,https://www.linkedin.com/pulse/querido-alumno-...,01/01/2023 15:32,01/01/2023 17:50,Querido alumno universitario de grado: Te esta...,Cada vez me siento más como un profesor del in...,725,16298,311,414,11,296,487
4,OYOYOY,Penetrator,youtube.com,https://www.youtube.com/watch?v=kEkSwIN6Yns,01/01/2023 09:34,01/01/2023 17:20,Encuentro con un joven zorro ártico en Groenla...,"Un joven zorro ártico salvaje, movido por la c...",234,3803,129,105,0,15,482
5,politica,Abajo,elviejotopo.com,https://www.elviejotopo.com/topoexpress/la-ult...,01/01/2023 11:02,01/01/2023 16:35,La ultraderecha como expresión del colapso en ...,Corría el año 2012. La crisis económica deriva...,350,2806,146,204,7,105,474
6,actualidad,kRaSh-mAn,elcomarcal.net,https://elcomarcal.net/algeciras/incendio-en-i...,01/01/2023 11:58,01/01/2023 15:55,Incendio en una petroquímica pone en alerta a ...,"En un comunicado del alcalde de Algeciras, Jos...",221,1638,103,118,0,15,542
7,cultura,ElLocoDelMolino,youtube.com,https://www.youtube.com/watch?v=UTxpNiA_Ujc,01/01/2023 12:56,01/01/2023 15:45,El Everest en 3D desde el campo base hasta la ...,Vídeo que muestra una ruta de escalada del Eve...,110,2372,69,41,1,17,552
8,actualidad,eaglesight1,eldiario.es,https://www.eldiario.es/economia/economia-espa...,01/01/2023 10:49,01/01/2023 15:35,La economía de España liderará el crecimiento ...,"Todas las predicciones, de las más optimistas ...",479,1278,212,267,9,216,428
9,cultura,ChanVader,bbc.com,https://www.bbc.com/mundo/noticias-64103773,01/01/2023 11:52,01/01/2023 14:25,Por qué Gengis Kan viajaba con una carreta lle...,El fundador del imperio contiguo más grande de...,163,7579,84,79,0,46,574


In [9]:

df.to_csv('meneame1.csv')


In [10]:

df.to_csv('meneame2.csv', index=False)


In [11]:

df.to_csv('meneame3.csv', sep=';', encoding='utf-8', index=False)
